In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [2]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

warnings.filterwarnings('ignore')

C:\Users\Anoshan\AppData\Local\Temp\ipykernel_19124\1588287611.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


In [9]:
path = "./Resources/Datasets"
SalesData = pd.read_csv(path + '/sales_data.csv')
WeekData = pd.read_csv(path + '/week_data.csv')
ProductData = pd.read_csv(path + '/product_data.csv')
OutletsData = pd.read_csv(path + '/outlets_data.csv')
FreezerData = pd.read_csv(path + '/freezer_data.csv')

In [11]:
WeekData['Week'] = WeekData['Week'].str.replace('Week ', '').astype(int)
#rename the column to 'week'
WeekData.rename(columns={'Week':'week'}, inplace=True)

In [12]:
CombinedData = pd.merge(SalesData, WeekData, left_on='week', right_on='week')
CombinedData = pd.merge(CombinedData, ProductData, on='pid')
CombinedData = pd.merge(CombinedData, OutletsData, on='Outlet_ID')


In [13]:
CombinedData.head()

,Outlet_ID,week,pid,product_name_x,no_units,Start Date,End Date,volume,product_name_y,price,area(sqft)
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,1/2/2023,1/8/2023,0.2,Vanilla Mini Cone,100,5660
1,ID7203,2,IP1,Vanilla Mini Cone,644.0,1/9/2023,1/15/2023,0.2,Vanilla Mini Cone,100,5660
2,ID7203,3,IP1,Vanilla Mini Cone,600.0,1/16/2023,1/22/2023,0.2,Vanilla Mini Cone,100,5660
3,ID7203,4,IP1,Vanilla Mini Cone,106.0,1/23/2023,1/29/2023,0.2,Vanilla Mini Cone,100,5660
4,ID7203,5,IP1,Vanilla Mini Cone,775.0,1/30/2023,2/5/2023,0.2,Vanilla Mini Cone,100,5660
